# Heat Pump

- Einleitung, Anwendungsfälle, Topologien, ...
- Definition von Aufwänden und Nutzen

## Exercise

- Bauen lassen der WP in TESPy mit Exergieanalyse
- Erstellen einer Exergetischen Auswertung im Diagramm (z. B. Wasserfall), ermitteln der großen Brocken
   - Analyse von Isentropenwirkungsgrad
   - Analyse von Temperaturdifferenzen
- Anheben/Absenken der Umgebungstemperaturniveaus -> Auswirkung auf Komponentenwirkungsgrade und Gesamtwirkungsgrad

## Theorie 2

- Verbesserung des COP -> Andere Topologie

## Übung 2

- Bauen der WP
- Warum verbesser es den Wirkungsgrad?
    - Verlgeich Aufwand und Nutzenströme bei gleichem Nennwärmestrom
    - Vergleich der Komponentenverluste
    - ...
